In [1]:
import pandas as pd
df = pd.read_csv("HR_capstone_dataset.csv")
print(df.head())

   satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                0.38             0.53               2                   157   
1                0.80             0.86               5                   262   
2                0.11             0.88               7                   272   
3                0.72             0.87               5                   223   
4                0.37             0.52               2                   159   

   time_spend_company  Work_accident  left  promotion_last_5years Department  \
0                   3              0     1                      0      sales   
1                   6              0     1                      0      sales   
2                   4              0     1                      0      sales   
3                   5              0     1                      0      sales   
4                   3              0     1                      0      sales   

   salary  
0     low  
1  medium  
2 

In [2]:
print(df['Department'].value_counts())

Department
sales          4140
technical      2720
support        2229
IT             1227
product_mng     902
marketing       858
RandD           787
accounting      767
hr              739
management      630
Name: count, dtype: int64


In [3]:
print(df['Department'].value_counts(normalize=True))

Department
sales          0.276018
technical      0.181345
support        0.148610
IT             0.081805
product_mng    0.060137
marketing      0.057204
RandD          0.052470
accounting     0.051137
hr             0.049270
management     0.042003
Name: proportion, dtype: float64


In [ ]:
department_counts_all = df['Department'].value_counts()
cumulative_percentage_all = (department_counts_all.cumsum() / department_counts_all.sum()) * 100
departments_80_percent_all = cumulative_percentage_all[cumulative_percentage_all <= 80]
department_info_all = {department: department_counts_all[department] for department in departments_80_percent_all.index}
department_info_all



{'sales': 4140,
 'technical': 2720,
 'support': 2229,
 'IT': 1227,
 'product_mng': 902}

In [ ]:
quantile_diff = {}
for department in department_info_all.keys():
    dept_data = df[df['Department'] == department]['average_montly_hours']
    quantile_diff[department] = dept_data.quantile(0.9) - dept_data.quantile(0.1)
department_to_remove = max(quantile_diff, key=quantile_diff.get)

department_to_remove


'technical'

In [ ]:
remaining_departments = {dept: quantile_diff[dept] for dept in quantile_diff if dept != department_to_remove}
department_to_remove_next = min(remaining_departments, key=remaining_departments.get)

department_to_remove_next


'product_mng'

In [ ]:
departments_remaining = [dept for dept in remaining_departments if dept != department_to_remove_next]
remaining_data = df[df['Department'].isin(departments_remaining)]

correlation_coefficients = {}
for department in departments_remaining:
    dept_data = remaining_data[remaining_data['Department'] == department]
    correlation = dept_data['number_project'].corr(dept_data['satisfaction_level'])
    correlation_coefficients[department] = abs(correlation)

department_highest_correlation = max(correlation_coefficients, key=correlation_coefficients.get)

department_highest_correlation


'IT'

In [ ]:
high_salary_data = remaining_data[remaining_data['salary'] == 'high']

median_satisfaction_high_salary = {}
for department in departments_remaining:
    dept_data = high_salary_data[high_salary_data['Department'] == department]
    median_satisfaction = dept_data['satisfaction_level'].median()
    median_satisfaction_high_salary[department] = median_satisfaction

department_highest_median_satisfaction = max(median_satisfaction_high_salary, key=median_satisfaction_high_salary.get)

department_highest_median_satisfaction


'sales'

In [29]:
print(median_satisfaction_high_salary)

{'sales': 0.69, 'support': 0.67, 'IT': 0.65}


In [27]:
print(departments_remaining)

['sales', 'support', 'IT']


In [ ]:
median_difference_high_salary = {}
for department in departments_remaining:
    dept_data = high_salary_data[high_salary_data['Department'] == department]
    if not dept_data.empty:
        median_diff = (dept_data['satisfaction_level'].median() - dept_data['last_evaluation'].median())
        median_difference_high_salary[department] = median_diff

department_highest_median_diff = max(median_difference_high_salary, key=median_difference_high_salary.get)
highest_median_diff_value = median_difference_high_salary[department_highest_median_diff]

f"{department_highest_median_diff}: {highest_median_diff_value:.4f}"


'sales: -0.0100'

In [30]:
median_difference_high_salary

{'sales': -0.010000000000000009,
 'support': -0.04999999999999993,
 'IT': -0.04999999999999993}